In [10]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/happy522/ChatGPT-as-a-CAT-tool-in-Easy-language-translation_Implemenatation/refs/heads/main/GPT_4_0/Matrix_Scores_GPT4_0.csv")
df

HTTPError: HTTP Error 404: Not Found

In [11]:
import pandas as pd
import numpy as np
from scipy import stats

# Define which metrics are higher-is-better
metrics_info = {
    'TTR': False,
    'LD': False,
    'WR': False,
    'MDD': False,
    'SI': False,
    'Cross': False,
    'WSF': False,
    'LIX': False,
    'FRE': True,
    'Fog': False
}

def compare_columns(col1, col2):
    """
    Compare two paired sets of values and compute:
    - Shapiro-Wilk p-value for normality of differences
    - Paired t-test or Wilcoxon test
    - Effect size (Cohen's d for t-test, rank-biserial for Wilcoxon)
    """
    diff = col1 - col2
    shapiro_p = stats.shapiro(diff).pvalue

    if shapiro_p > 0.05:
        # Normal differences → paired t-test
        p_val = stats.ttest_rel(col1, col2).pvalue
        effect = diff.mean() / np.std(diff, ddof=1)  # Cohen's d
        test = "t-test"
    else:
        # Non-normal differences → Wilcoxon signed-rank test
        p_val = stats.wilcoxon(col1, col2).pvalue
        effect = (np.sum(diff > 0) - np.sum(diff < 0)) / len(diff)  # Rank-biserial correlation
        test = "Wilcoxon"

    return shapiro_p, p_val, effect, test

# Prepare results
results = []

for metric, higher_better in metrics_info.items():
    hol = df[f"{metric}_Holistic"]
    lin = df[f"{metric}_Linguistic"]
    org = df[f"{metric}_Original"]

    shapiro_p, p_val, effect, test = compare_columns(hol, lin)

    # Determine which column is better based on direction
    if higher_better:
        winner = "Holistic" if hol.mean() > lin.mean() else "Linguistic"
    else:
        winner = "Holistic" if hol.mean() < lin.mean() else "Linguistic"

    results.append({
        "Metric": metric,
        "Original_mean": round(org.mean(), 3),
        "Holistic_mean": round(hol.mean(), 3),
        "Linguistic_mean": round(lin.mean(), 3),
        "Shapiro_p": round(shapiro_p, 4),
        "p_value": round(p_val, 4),
        "Effect_size": round(effect, 2),
        "Test": test,
        "Better": winner
    })

# Convert to DataFrame and save
results_df = pd.DataFrame(results)
results_df.to_csv("Holistic_vs_Linguistic_4_0.csv", index=False)
results_df


,Metric,Original_mean,Holistic_mean,Linguistic_mean,Shapiro_p,p_value,Effect_size,Test,Better
0,TTR,0.555,0.666,0.559,0.1667,0.0000,1.20,t-test,Linguistic
1,LD,0.504,0.471,0.475,0.4044,0.7024,-0.09,t-test,Holistic
2,WR,0.492,0.413,0.446,0.2770,0.0044,-0.72,t-test,Holistic
3,MDD,2.797,2.491,2.858,0.5596,0.0009,-0.88,t-test,Holistic
4,SI,0.304,0.425,0.454,0.4986,0.3634,-0.21,t-test,Holistic
5,Cross,4.550,0.900,3.900,0.0027,0.0005,-0.75,Wilcoxon,Holistic
6,WSF,6.128,5.456,6.843,0.0025,0.0008,-0.90,Wilcoxon,Holistic
7,LIX,49.579,43.698,52.658,0.0205,0.0002,-0.90,Wilcoxon,Holistic
8,FRE,40.368,57.057,42.829,0.0073,0.0001,0.90,Wilcoxon,Holistic
9,Fog,15.704,13.922,17.140,0.0016,0.0003,-0.90,Wilcoxon,Holistic
